In [1]:
import numpy as np 
import pandas as pd
from keras.models import Sequential,Model
import tensorflow as tf
from keras.layers import Dense, Activation, Input,RepeatVector,Embedding, Flatten, Concatenate,Dropout
from keras.models import Model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from sklearn import metrics as mt
from keras import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import average, concatenate,RepeatVector,Lambda,add,subtract
from keras.layers.normalization import BatchNormalization
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from sklearn.utils import class_weight, shuffle
import random
import os
from keras import backend as K
from keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau,CSVLogger)
from sklearn.metrics import precision_recall_fscore_support, classification_report
import pickle
from keras.layers import Conv2D, MaxPooling2D,Conv1D,GlobalMaxPooling1D,MaxPooling1D,Reshape,Add
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D,AveragePooling1D
from keras.regularizers import l2 
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
def scroe(y_true,y_pred):
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))
    print("ACC:",accuracy_score(y_true, y_pred))

#Prepare Data

# with open('final_data_dude_rd2.npy', 'rb') as handle:
data = np.load('final_data_dude_rd2_opp.npy').item()

In [3]:
X, proteins, ic50 = data['X'], data['proteins'], data['ic50']
print(len(X))
k = sum(ic50)

print ('Class 1:',k)

td = {}

pr = data['pr_names']

for i in pr:
	if i not in td:
		td[i] = 1
	else:
		td[i] += 1
li = [[td[i], i] for i in td]
li.sort(reverse = True)
print (li)
li = [i[1] for i in li]

print (len(li))
#train = li[:10] + list(random.choices(li[10:], k = 62))
train = list(random.choices(li, k = 72))
print (len(train))
#test = list(set(np.unique(pr)) - set(train))
fac = []
fac_t = []
for i in range (len(pr)):
	if pr[i] in train:
		fac.append(i)
	else:
		fac_t.append(i)
fac = np.array(fac)
fac_t = np.array(fac_t)

proteins = np.array(proteins)
X = np.array(X)
ic50 = np.array(ic50)

print (proteins.shape, X.shape, ic50.shape)
print("Class 0:",(len(ic50)-sum(ic50)) )


# In[5]:

a, b = sum(ic50), len(ic50)-sum(ic50)
print('ratio:',sum(ic50)/(len(ic50)-sum(ic50)))

45609
Class 1: 22805
[[1184, '3E37'], [1156, '2QD9'], [1144, '830C'], [1084, '2RGP'], [1074, '3KL6'], [1072, '1XL2'], [1066, '2I78'], [1064, '2OI0'], [1048, '3EL8'], [1016, '3L3M'], [984, '1BCD'], [968, '2GTK'], [964, '3EML'], [960, '3PBL'], [948, '1H00'], [922, '1YPE'], [906, '1E66'], [898, '2AYW'], [870, '3LN1'], [840, '2OF2'], [818, '2P2I'], [796, '1UDT'], [766, '1SJ0'], [746, '2P54'], [734, '2FSZ'], [676, '3LAN'], [660, '3FRJ'], [586, '3KBA'], [586, '3CQW'], [566, '3L5D'], [564, '3BKL'], [538, '2AM9'], [515, '3BQD'], [494, '2VT4'], [480, '2ZNP'], [462, '3NY8'], [462, '3NXO'], [398, '2CNK'], [390, '2OYU'], [370, '3MAX'], [364, '2HZI'], [342, '3CHP'], [340, '3NXU'], [340, '3F07'], [340, '3CCW'], [332, '3LQ8'], [332, '3KRJ'], [332, '3G0E'], [324, '1SQT'], [318, '2HV5'], [316, '3KGC'], [304, '3D4Q'], [296, '2ZEC'], [296, '2OJ9'], [278, '3C4F'], [276, '2ICA'], [270, '2I0E'], [266, '3HMM'], [262, '1MV9'], [260, '2AZR'], [244, '1S3B'], [242, '3EQH'], [240, '1R9O'], [234, '3D0E'], [232, '3

In [4]:
Proeins_shape=proteins.shape[1:]
Drags_shape=X.shape[1]
print(Proeins_shape)
print(Drags_shape)

train_X = X[fac]
valid_X = X[fac_t]

train_proteins = proteins[fac]
valid_proteins = proteins[fac_t]

train_y = ic50[fac]
valid_y = ic50[fac_t]

# In[11]:

print (train_X.shape, valid_X.shape)

(2, 9000)
512
(22921, 512) (22688, 512)


In [5]:
def get_callback_list_by_model(model_name):
 
    checkpoint = ModelCheckpoint('model_zoo/'+model_name+'.h5', monitor='val_loss', verbose=1, 
                                 save_best_only=True, mode='min', save_weights_only = True)
    reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, 
                                       verbose=1, mode='auto', epsilon=0.0001)
    early = EarlyStopping(monitor="val_loss", 
                          mode="min", 
                          patience=7)
    csv_logger = CSVLogger(filename='./training_log_'+model_name+'.csv',
                       separator=',',
                       append=True)
    
    callbacks_list = [checkpoint,reduceLROnPlat, early,csv_logger]
    return callbacks_list

In [6]:
def evaluate_model(model,model_name,Epoch):
    callbacks_list = get_callback_list_by_model(model_name)
#     train_proteins_shuffle, train_X_shuffle, train_y_shuffle = shuffle(train_proteins, train_X, train_y,random_state=randomS)
    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['accuracy'])

    model.fit(x=[train_proteins,train_X], # create a list of inputs for embeddings
            y=train_y, epochs=Epoch, 
            batch_size=64, verbose=1,
            validation_data = ([valid_proteins,valid_X],valid_y),
            callbacks=callbacks_list         
               )

    re=np.round(np.squeeze(model.predict([valid_proteins,valid_X])))
    scroe(re,list(valid_y))

In [7]:
def conv_blocks(ft_number,k_size,input_tensor):
    x = Conv1D(filters=ft_number, 
                     kernel_size=k_size, 
                     padding='same',
                     kernel_regularizer=l2(l2_lambda)
              )(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    
#     x = Conv1D(filters=ft_number, 
#                      kernel_size=k_size, 
#                      padding='same',
#                      kernel_regularizer=l2(l2_lambda)
#               )(x)
#     x = Activation('relu')(x)
#     x = BatchNormalization()(x)
    
    return x
def op(inputs):
    x, y = inputs
    return K.pow((x - y), 2) 

In [8]:
def conv_branch(init_input,kernel_size):
    x=conv_blocks(ft_number=32,k_size=kernel_size,input_tensor=init_input)
    x=MaxPooling1D(2,padding='same')(x)

    x=conv_blocks(ft_number=64,k_size=kernel_size,input_tensor=x)
    x=MaxPooling1D(2,padding='same')(x)

    x=conv_blocks(ft_number=128,k_size=kernel_size,input_tensor=x)
    u = GlobalMaxPooling1D()(x)
    u_broadcast=RepeatVector(x.shape[1])(u)

    o=Lambda(op)([u_broadcast,x])  # K.pow((x - y), 2) 
    var = GlobalAveragePooling1D()(o)
    X_vector = concatenate([u,var])
    return X_vector

In [9]:
l2_lambda=0.05
def create_multiBranch_Conv_model(Proeins_shape,Drags_shape):
    
    # left branch --> dimension reduction for Proeins_shape
    
    proeins_input_tensor = Input(shape=Proeins_shape, name='proeins_input_tensor')
    init_input = Reshape((Proeins_shape[1], Proeins_shape[0]),input_shape=Proeins_shape,name='init_input')(proeins_input_tensor)

    #branch 0
    w=conv_branch(init_input,10)
    #branch 1
    x=conv_branch(init_input,5)

    #branch 2
    y=conv_branch(init_input,15)
    
    #branch 3
    z=conv_branch(init_input,20)
    
    protein_concat = concatenate([w,x,y,z], name='protein_concat_')
    
    protein_dense = Dense(128)(protein_concat)
    protein_dense = BatchNormalization()(protein_dense)
    protein_dense = Activation('relu')(protein_dense)
    protein_dense = Dropout(0.5)(protein_dense)
    
    
    # right branch --> dimension reduction for drug/ligand
    drag_input_tensor = Input(shape=(Drags_shape,),name='drag_input_tensor')
    d = Dense(128)(drag_input_tensor)
    d = BatchNormalization()(d)
    d = Activation('relu')(d)
    d = Dropout(0.5)(d)

    
#     # merge the branches together
    final_branch = concatenate([protein_dense,d], name='protein_darg_concat_')
    
    final_dense = Dense(64)(final_branch)
    final_dense = BatchNormalization()(final_dense)
    final_dense = Activation('relu')(final_dense)
    final_dense = Dropout(0.5)(final_dense)

#     final_dense = Dense(32)(final_dense)
#     final_dense = BatchNormalization()(final_dense)
#     final_dense = Activation('relu')(final_dense)
#     final_dense = Dropout(0.5)(final_dense)
    
    final_output = Dense(1, activation='sigmoid', name='final_output')(final_dense)
    
#     model = Model(inputs=proeins_input_tensor, outputs=protein_concat)
    model = Model(inputs=[proeins_input_tensor,drag_input_tensor], outputs=final_output)
    return model

In [10]:
multiBranch_Conv_model = create_multiBranch_Conv_model(Proeins_shape,Drags_shape,)
evaluate_model(multiBranch_Conv_model,'multiBranch_Conv_model',20)
# SVG(model_to_dot(multiBranch_Conv_model,show_shapes=True).create(prog='dot', format='svg'))

/conda/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 22921 samples, validate on 22688 samples
Epoch 1/20
22921/22921 [==============================] - 120s 5ms/step - loss: 2.0221 - acc: 0.7862 - val_loss: 0.1645 - val_acc: 0.8517

Epoch 00001: val_loss improved from inf to 0.16451, saving model to model_zoo/multiBranch_Conv_model.h5
Epoch 2/20
22921/22921 [==============================] - 102s 4ms/step - loss: 0.1326 - acc: 0.8904 - val_loss: 0.1354 - val_acc: 0.8622

Epoch 00002: val_loss improved from 0.16451 to 0.13540, saving model to model_zoo/multiBranch_Conv_model.h5
Epoch 3/20
22921/22921 [==============================] - 102s 4ms/step - loss: 0.1102 - acc: 0.9091 - val_loss: 0.1272 - val_acc: 0.8684

Epoch 00003: val_loss improved from 0.13540 to 0.12724, saving model to model_zoo/multiBranch_Conv_model.h5
Epoch 4/20
22921/22921 [==============================] - 102s 4ms/step - loss: 0.0966 - acc: 0.9224 - val_loss: 0.1221 - val_acc: 0.8757

Epoch 00004: val_loss improved from 0.12724 to 0.12210, saving model to mo

In [11]:
multiBranch_Conv_model.load_weights('model_zoo/multiBranch_Conv_model.h5')
re1 =np.squeeze(multiBranch_Conv_model.predict([valid_proteins,valid_X]))
print (roc_auc_score(valid_y, re1))

0.9648677735295843


In [12]:
# result = []
# reg=[0.001,0.01,0.1,0.05]
# window_size_search=[[30,15,5],[30,30,30],[5,5,5],[10,10,10],[20,10,10],[20,20,20]]

In [13]:
# def get_model_auc_byWindow(windows):
#     Conv_model = create_Conv_model(Proeins_shape,Drags_shape,windows)
#     evaluate_model(Conv_model,'Conv_model',20)
#     Conv_model.load_weights('model_zoo/Conv_model.h5')
#     re1 =np.squeeze(Conv_model.predict([valid_proteins,valid_X]))
#     print ("For Window",windows, "Reg is ",l2_lambda,roc_auc_score(valid_y, re1))
#     result.append(roc_auc_score(valid_y, re1))

In [14]:
# for w in window_size_search:
#     get_model_auc_byWindow(w)
    

In [15]:
# Reresult = []
# for R in reg:
#     l2_lambda=R
#     get_model_auc_byWindow([20,20,20])
    